# 7: Diminishing OVTT by Distance

The primary shortcoming of the specification in Model 6 is that the estimated value of IVT is unrealistically small. At least two alternatives can be considered for getting an improved estimate of the value of out-of-vehicle time. One is to assume that the sensitivity of travelers to OVT diminishes with the trip distance. The idea behind this is that travelers are more willing to tolerate higher out-of-vehicle time for a long trip rather than for a short trip.

Model 7 ensures this result by including total travel time (the sum of in-vehicle and out-of-vehicle time) and out-of-vehicle time divided by distance in place of in- and out-of-vehicle travel time. (pp. 114)

In [ ]:
import larch

In [ ]:
larch.__version__

This example is a mode choice model built using the MTC example dataset. First we create the DB and Model objects:

In [ ]:
d = larch.examples.MTC(format="dataset")
d

In [ ]:
m = larch.Model(d, compute_engine="numba")

Then we can build up the utility function. We’ll use some :ref:idco data first, using the Model.utility.co attribute. This attribute is a dict-like object, to which we can assign :class:LinearFunction objects for each alternative code.

In [ ]:
from larch import PX, P, X

m.utility_co[2] = P("ASC_SR2") + P("hhinc#2,3") * X("hhinc")
m.utility_co[3] = P("ASC_SR3P") + P("hhinc#2,3") * X("hhinc")
m.utility_co[4] = P("ASC_TRAN") + P("hhinc#4") * X("hhinc")
m.utility_co[5] = P("ASC_BIKE") + P("hhinc#5") * X("hhinc")
m.utility_co[6] = P("ASC_WALK") + P("hhinc#6") * X("hhinc")

Next we’ll use some idca data, with the utility_ca attribute. This attribute is only a single :class:LinearFunction that is applied across all alternatives using :ref:idca data. Because the data is structured to vary across alternatives, the parameters (and thus the structure of the :class:LinearFunction) does not need to vary across alternatives.

We diminish the effect of ovtt by dividing its parameter by distance.

In [ ]:
m.utility_ca = (
    +PX("totcost")
    + P("motorized_time") * X("(altid <= 4) * tottime")
    + P("nonmotorized_time") * X("(altid > 4) * tottime")
    + P("motorized_ovtbydist") * X("(altid <= 4) * ovtt/dist")
)

Lastly, we need to identify idca Format data that gives the availability for each alternative, as well as the number of times each alternative is chosen. (In traditional discrete choice analysis, this is often 0 or 1, but it need not be binary, or even integral.)


In [ ]:
m.availability_ca_var = "avail"
m.choice_ca_var = "chose"

And let’s give our model a descriptive title.

In [ ]:
m.title = "MTC Example 7, Diminishing OVTT by Distance"

We can view a summary of the choices and alternative availabilities to make sure the model is set up correctly.

In [ ]:
m.choice_avail_summary()

We’ll set a parameter cap (bound) at +/- 20, which helps improve the numerical stability of the optimization algorithm used in estimation.

In [ ]:
m.set_cap(20)

Having created this model, we can then estimate it:

In [ ]:
assert m.compute_engine == "numba"

In [ ]:
result = m.maximize_loglike(stderr=True)
m.calculate_parameter_covariance()
m.loglike()

In [ ]:
m.parameter_summary()

It is a little tough to read this report because the parameters show up in alphabetical order. We can use the reorder method to fix this and group them systematically:

In [ ]:
m.ordering = (
    (
        "LOS",
        ".*cost.*",
        ".*time.*",
        ".*dist.*",
    ),
    (
        "Income",
        "hhinc.*",
    ),
    (
        "ASCs",
        "ASC.*",
    ),
)

In [ ]:
m.parameter_summary()

Finally, let's print model statistics.  Note that if you want LL at constants you need to run a separate model.

In [ ]:
m.estimation_statistics()

In [ ]:
# TEST
revealed_x = dict(zip(m.pnames, result.x))

In [ ]:
# TEST
from pytest import approx

expected_x = {
    "ASC_BIKE": -2.675340279013382,
    "ASC_SR2": -2.1871708718901455,
    "ASC_SR3P": -3.5170483581773535,
    "ASC_TRAN": -0.039611848334672385,
    "ASC_WALK": -1.0206078711851112,
    "hhinc#2,3": -0.001383608162986253,
    "hhinc#4": -0.007257729971526785,
    "hhinc#5": -0.01196363695212793,
    "hhinc#6": -0.008182417861122782,
    "motorized_ovtbydist": -0.18145744778558676,
    "motorized_time": -0.04156509908994426,
    "nonmotorized_time": -0.04755705411055601,
    "totcost": -0.004114472584806616,
}
for k in expected_x:
    assert revealed_x[k] == approx(expected_x[k], 2e-2), (
        f"{k}, {revealed_x[k] / expected_x[k]}"
    )